# Working with Planet Data in Python

In this notebook we'll use the planet and stactools libraries to perform the following:
- Search for PlanetScope scenes
- Download orders from PlanetScope scenes for visual and analytic products
- Convert those orders into STACs
- Merge STACs from different orders
- Modify the layout of STACs based on item properties.

You'll need the `planet python client <https://planetlabs.github.io/planet-client-python/cli/index.html>`_ installed, as well as the ``stactools_planet`` stactools subpackage.

You'll also need an account with access to the orders API.
This tutorial will use data that is available in the [Developer Trial Program](https://developers.planet.com/devtrial), so sign up there if you don't already have an account - and let them know you are creating STACs!

First we'll set up some of the directories that we'll be using:

In [9]:
import os

In [10]:
DATA_DIR = '/opt/src'
DOWNLOAD_DIR = os.path.join(DATA_DIR, 'order-downloads')
STAC_DIR = os.path.join(DATA_DIR, 'planet-stacs-2')

In [11]:
VISUAL_ORDER_DIR = os.path.join(DOWNLOAD_DIR, 'visual')
if not os.path.isdir(VISUAL_ORDER_DIR):
    os.makedirs(VISUAL_ORDER_DIR)

In [12]:
ANALYTIC_ORDER_DIR = os.path.join(DOWNLOAD_DIR, 'analytic')
if not os.path.isdir(ANALYTIC_ORDER_DIR):
    os.makedirs(ANALYTIC_ORDER_DIR)

In [5]:
TIMERANGE_ORDER_DIR = os.path.join(DOWNLOAD_DIR, 'range')
if not os.path.isdir(TIMERANGE_ORDER_DIR):
    os.makedirs(TIMERANGE_ORDER_DIR)

## Ordering from Planet's Orders API

We'll fetch some data over southern Myanmar using the planet client's search:

In [14]:
from planet import api
from planet.api import filters

client = api.ClientV1()

In [17]:
aoi = {
  "type": "Polygon",
  "coordinates": [
          [
            [
              95.18348693847656,
              15.792914297218834
            ],
            [
              95.36613464355469,
              15.792914297218834
            ],
            [
              95.36613464355469,
              15.983113625840078
            ],
            [
              95.18348693847656,
              15.983113625840078
            ],
            [
              95.18348693847656,
              15.792914297218834
            ]
          ]
        ]
}


In [18]:
query = filters.and_filter(
    filters.geom_filter(aoi),
    filters.range_filter('cloud_cover', gt=0.01),
    filters.range_filter('cloud_cover', lt=0.3),
    filters.date_range('acquired', gt='2019-01-01'),
    filters.date_range('acquired', lt='2019-01-13')
)

request = filters.build_search_request(
    query, item_types=['PSScene3Band']
)

In [19]:
result = client.quick_search(request)

In [20]:
items = [i for i in result.items_iter(limit=None)]

In [21]:
len(items)

8

We have 8 items from this search. We'll grab the IDs from them to use in our order.


In [22]:
ids = [i['id'] for i in items]
ids

['20190111_034458_0f3f',
 '20190111_034459_0f3f',
 '20190111_034457_0f3f',
 '20190111_033800_0f46',
 '20190111_033759_0f46',
 '20190109_034416_103d',
 '20190109_034415_103d',
 '20190109_034414_103d']

### Ordering visual products

Now we'll use the planet client's order functionality to generate an order for the PSScene3Band visual product bundles for those items:

In [23]:
order_request = {
    'name': 'My order - visual cogs',
    'products': [
        {
            'item_ids': ids,
            'item_type': 'PSScene3Band',
            'product_bundle': 'visual'
        }
    ],
    'tools': [
        {
          'file_format': {
            'format': 'COG'
          }
        }
      ],
    'delivery': { 'single_archive': True },
    'notifications': { 'email': True }
}

order_request

{'name': 'My order - visual cogs',
 'products': [{'item_ids': ['20190111_034458_0f3f',
    '20190111_034459_0f3f',
    '20190111_034457_0f3f',
    '20190111_033800_0f46',
    '20190111_033759_0f46',
    '20190109_034416_103d',
    '20190109_034415_103d',
    '20190109_034414_103d'],
   'item_type': 'PSScene3Band',
   'product_bundle': 'visual'}],
 'tools': [{'file_format': {'format': 'COG'}}],
 'delivery': {'single_archive': True},
 'notifications': {'email': True}}

In [24]:
order = client.create_order(order_request)
order_id = order.get()['id']

You can check your order state with the next cell. You should get an email when it's ready, and when it is this should show `success`.

In [28]:
[order['state'] 
 for order in client.get_orders().get()['orders'] 
 if order['id'] == order_id]

['success']

Once the order has succeeded we can download it:

In [29]:
processed_order = client.get_individual_order(order_id)

In [32]:
def download_order(order, target_dir):
    for result in order.get_results():
        fname = os.path.basename(result['name'])
        # We only want the zipped order
        if fname.endswith('.zip'):
            output_path = os.path.join(target_dir, fname)
            location = result['location']
            print('Downloading {}'.format(fname))
            !wget -O $output_path $location
            !unzip -o -d $target_dir $output_path

In [33]:
download_order(processed_order, VISUAL_ORDER_DIR)

--2020-10-30 23:56:12--  https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE2MDQxODgyMTgsInN1YiI6InQzajltRHdKeHBwT010cXBtQmliUkFiaWJNbm5uVnFGdmVPSVhjblVDSnhtVGsySnVxSHJXaWJDd0YvRy9RblcycHViUkxvZ0R5d2tHSi8xRnR2VllnPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS8xN2Q2OGJhMC05YjFhLTQzMDktYTI0Zi01MGNlNWEwMDQzYjIvb3V0cHV0LnppcD9FeHBpcmVzPTE2MDQxODgyMThcdTAwMjZHb29nbGVBY2Nlc3NJZD1jb21wdXRlLWdjcy1zdmNhY2MlNDBwbGFuZXQtY29tcHV0ZS1wcm9kLmlhbS5nc2VydmljZWFjY291bnQuY29tXHUwMDI2U2lnbmF0dXJlPWY1dzdidW4lMkZTdDdBemhKQ2l0VVdYY0N6MUc0YmVrRWxQanp3bVdvTXR0TUlBMFd6ZVRPQ0xwJTJCcWFRdkJDWnhDbkVGWktYMnVnY3pQeVFzaGFOaW9zJTJCQzBLV3pwViUyQndHWXpjOG9SQlJ0RG43SUtlZW5BaElFNEI1YngxdFElMkJaUnZVT3A0QkNDYlFBMW9ITnNTUFIwb0hwelJzZyUyRk9iTTgyWjBnSzNKS2g3R3J5akRsM0wwbWNnY3drWXh0bzJlZUdmVnB6QjB2aUdodFgwSnF5Wm9

### Ordering analytic products

We can similarly order PSScene4Band surface reflectance products:

In [34]:
analytic_order_request = {
    'name': 'My order - analytic surface reflectance cogs',
    'products': [
        {
            'item_ids': ids,
            'item_type': 'PSScene4Band',
            'product_bundle': 'analytic_sr'
        }
    ],
    'tools': [
        {
          'file_format': {
            'format': 'COG'
          }
        }
      ],
    'delivery': { 'single_archive': True },
    'notifications': { 'email': True }
}

In [35]:
order = client.create_order(order_request)
order_id = order.get()['id']

In [39]:
[order['state'] 
 for order in client.get_orders().get()['orders'] 
 if order['id'] == order_id]

['success']

Once again, when our order succeeds, we'll download the order:

In [40]:
processed_order = client.get_individual_order(order_id)

In [41]:
download_order(processed_order, ANALYTIC_ORDER_DIR)

--2020-10-31 00:10:14--  https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE2MDQxODk0MTIsInN1YiI6IjNRYUxVYUVLWU9uNEwwRU9QdFZKaTBsVEg2ZlhYaDhzVlVhcDFGWVR3TENtRDRMM0p5RDFaRVVCNU5icU94aFhOR0MvNjJwN0hVcS8yTFNrWC8vZzFBPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS85MjcxZjFjNC1mMmRmLTQ3NTAtOTU5OC1mZWNkODAwYTU3MGYvb3V0cHV0LnppcD9FeHBpcmVzPTE2MDQxODk0MTJcdTAwMjZHb29nbGVBY2Nlc3NJZD1jb21wdXRlLWdjcy1zdmNhY2MlNDBwbGFuZXQtY29tcHV0ZS1wcm9kLmlhbS5nc2VydmljZWFjY291bnQuY29tXHUwMDI2U2lnbmF0dXJlPVZnRGJaUVhZaiUyRkc2TzVyMXFYUyUyQldQU3RVJTJCJTJGbzI3ekIzZyUyRkFvbjhDcDR5eGt4ZSUyRnBaWWd5OG9pd1lpJTJGbWtXN2JKVnpWT0RxdTFPQUQzZHdjZ09tUmNOJTJGeXglMkZzek9ncTJEQXJqVndQY045NG8wUFBlWkRIJTJCaiUyRkNKJTJGZGQxMFEyV3o0S0ZqbCUyQng1dFNMSlJabXVUWnRvR2NQYVlSc0JuU05KUlZnT0lYelZjMyUyRjB4TWtNWW4wZExqQ1F4U3RaUWgwQXV

## Turning orders into STAC

We can create STACs of each of these orders with the `stactools.planet` subpackage. To do so, we read in the manifests of the order into `OrderManifest` objects, and convert them to STAC PySTAC collections:

In [42]:
from stactools.planet import OrderManifest

In [43]:
visual_manifest = OrderManifest.from_file(os.path.join(VISUAL_ORDER_DIR, 'manifest.json'))

Now we use the `to_stac` method to create a PySTAC Collection:

In [44]:
visual_collection = visual_manifest.to_stac(
    collection_id='visual-cogs', 
    description='A planet order converted to STAC. Created in Python.',
    title='Planet data over S Myanmar'
)

We can save off our catalog, and then use stactools to copy the assets into our STAC. We'll save the catalog before copying to ensure the directory structure of our new STAC exists.

In [45]:
import pystac
import stactools.core

In [46]:
visual_collection.normalize_hrefs(os.path.join(STAC_DIR, 'visual'))
visual_collection.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

In [47]:
visual_collection = stactools.core.move_all_assets(visual_collection, copy=True, ignore_conflicts=True)

We now have a STAC that can be browsed with `stac browse`. See the CLI tutorial for steps on how to browse the catalog.

We can do the same thing with the analytic order:

In [48]:
analytic_manifest = OrderManifest.from_file(os.path.join(ANALYTIC_ORDER_DIR, 'manifest.json'))
analytic_collection = analytic_manifest.to_stac(
    collection_id='analytic-cogs', 
    description='A planet order converted to STAC. Created in Python.',
    title='Planet data over S Myanmar'
)
analytic_collection.normalize_hrefs(os.path.join(STAC_DIR, 'analytic'))
analytic_collection.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

analytic_collection = stactools.core.move_all_assets(analytic_collection, copy=True, ignore_conflicts=True)

### Merging STACs

Since the analytic and visual orders reference the same items, we may want to have them in a single catalog, with Items having both visual and analytic assets. We can use stactools's `merge_all_items` to merge the items from the analytic collection into the visual one:

In [49]:
visual_collection = stactools.core.merge_all_items(
    analytic_collection, 
    visual_collection,
    move_assets=True, 
    ignore_conflicts=True)

Here we used `move_assets` to move the assets from the analytic STAC next to the items created in the visual STAC. `ignore_conflicts` is needed here since both versions of the Item will have a metadata.json asset from the order, which should be the same metadata referencing the Planet Item.

We can get an item out of our visual collection and see it now has analytic assets as well:

In [50]:
item = next(visual_collection.get_all_items())
for key in item.assets:
    print(key)

visual:visual_xml
visual
metadata


In [51]:
visual_collection.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

### Adding more orders and items

We may want to merge in data that isn't different assets of our original items. To work with this, we'll create another order; this time, we'll use a slightly different area, and spread items over time:

In [52]:
aoi2 = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          94.75570678710938,
          15.695764366303791
        ],
        [
          94.93560791015625,
          15.695764366303791
        ],
        [
          94.93560791015625,
          15.855673509998681
        ],
        [
          94.75570678710938,
          15.855673509998681
        ],
        [
          94.75570678710938,
          15.695764366303791
        ]
      ]
    ]
  }

In [53]:
ids = []

months = ['2018-09',
          '2018-10',
          '2018-11',
          '2018-12',
          '2019-01',
          '2019-02',
          '2019-03']
for month in months:
    print('Searching for scenes in {}'.format(month))
    query = filters.and_filter(
        filters.geom_filter(aoi2),
        filters.range_filter('cloud_cover', gt=0.01),
        filters.range_filter('cloud_cover', lt=0.1),
        filters.date_range('acquired', gt='{}-01'.format(month)),
        filters.date_range('acquired', lt='{}-28'.format(month))
    )

    request =filters.build_search_request(
        query, item_types=['PSScene3Band']
    )
    
    result = client.quick_search(request)
    
    item = next(result.items_iter(limit=None))
    ids.append(item['id'])

Searching for scenes in 2018-09
Searching for scenes in 2018-10
Searching for scenes in 2018-11
Searching for scenes in 2018-12
Searching for scenes in 2019-01
Searching for scenes in 2019-02
Searching for scenes in 2019-03


We'll once again create the order, wait for it to succeed, download and unzip it:

In [54]:
order_request = {
    'name': 'My order - visual cogs',
    'products': [
        {
            'item_ids': ids,
            'item_type': 'PSScene3Band',
            'product_bundle': 'visual'
        }
    ],
    'tools': [
        {
          'file_format': {
            'format': 'COG'
          }
        }
      ],
    'delivery': { 'single_archive': True },
    'notifications': { 'email': True }
}

order_request

{'name': 'My order - visual cogs',
 'products': [{'item_ids': ['20180924_034401_0f3f',
    '20181025_034420_0f4e',
    '20181125_034649_0f28',
    '20181227_034225_0f2b',
    '20190119_034511_1035',
    '20190212_033542_1054',
    '20190322_034910_0f12'],
   'item_type': 'PSScene3Band',
   'product_bundle': 'visual'}],
 'tools': [{'file_format': {'format': 'COG'}}],
 'delivery': {'single_archive': True},
 'notifications': {'email': True}}

In [55]:
order = client.create_order(order_request)
order_id = order.get()['id']

In [60]:
[order['state'] 
 for order in client.get_orders().get()['orders'] 
 if order['id'] == order_id]

['success']

In [61]:
processed_order = client.get_individual_order(order_id)
download_order(processed_order, TIMERANGE_ORDER_DIR)

--2020-10-31 00:21:57--  https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE2MDQxOTAxMTYsInN1YiI6IkF5Sk1HcEtsSTQyTkhCbDBvWENSM0trZ1F4ZjR4TGpoV1ROeEg4S1h5ZUdSeHpua21KOG1USDRGa0xkbFZ4R09IbkVYS0lmY2JsOUJBT25PK2ZWZkhBPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS8zZjA5ZGQxOC05ZTRhLTQyNWQtYmM0Yy00ZWVhNDJhYjU0ZmIvb3V0cHV0LnppcD9FeHBpcmVzPTE2MDQxOTAxMTZcdTAwMjZHb29nbGVBY2Nlc3NJZD1jb21wdXRlLWdjcy1zdmNhY2MlNDBwbGFuZXQtY29tcHV0ZS1wcm9kLmlhbS5nc2VydmljZWFjY291bnQuY29tXHUwMDI2U2lnbmF0dXJlPU5yNUR4MXhlcnFYQ0dBak42JTJCa1VWMFVlS3lrRm1KMnNGMG5FQzlaeTFVOHo5aEdDSVZJViUyRkUlMkZkQTY2M0ZFZiUyQmppayUyRkM4UFdnUlBCM01RVGlGQVVjTjM0JTJGcXJlQWtPS3lQMXh0VzI0RGtMd1hneVpSMkMwM1JaYUFxZE5yc0tlQ25wbFUlMkZkd2wlMkIlMkJFSzhZMXN4cVVGSElUcVZJenZoQmttM3RpZ2lGN1lWbDNlemxHODNyQXBnbExZZWpCNW9SSlElMkIxdlJ4U1pKdXI

We can get a STAC of the order and merge it into our existing STAC:

In [63]:
range_manifest = OrderManifest.from_file(os.path.join(TIMERANGE_ORDER_DIR, 'manifest.json'))
range_collection = range_manifest.to_stac(collection_id='range_collection')

In [64]:
merged_collection = stactools.core.merge_all_items(
    range_collection, 
    visual_collection, 
    move_assets=False,
    ignore_conflicts=True)

Notices that we aren't moving the assets yet; we'll do that once the catalog is in it's final form.

Using the describe method, we see that we have our new items:

In [24]:
merged_collection.describe()

* <Collection id=visual-cogs>
  * <Item id=20190109_034414_103d>
  * <Item id=20190111_033800_0f46>
  * <Item id=20190111_033759_0f46>
  * <Item id=20190109_034416_103d>
  * <Item id=20190111_034459_0f3f>
  * <Item id=20190109_034415_103d>
  * <Item id=20190111_034458_0f3f>
  * <Item id=20190111_034457_0f3f>
  * <Item id=20180924_034401_0f3f>
  * <Item id=20190212_033542_1054>
  * <Item id=20190119_034511_1035>
  * <Item id=20181025_034420_0f4e>
  * <Item id=20181125_034649_0f28>
  * <Item id=20190322_034910_0f12>
  * <Item id=20181227_034225_0f2b>


## Modifying the layout of the catalog

Our catalog is still pretty small, but you can imagine in a larger catalog wanting to create some subdirectories and subcatalogs to organize the items into smaller groups. We can do that here with the PySTAC. We use a string template to generate subcatalogs based on the item's year and month:

In [67]:
merged_collection.generate_subcatalogs(template='{year}/{month}')

[<Catalog id=2019>,
 <Catalog id=1>,
 <Catalog id=2018>,
 <Catalog id=10>,
 <Catalog id=11>,
 <Catalog id=3>,
 <Catalog id=9>,
 <Catalog id=2>,
 <Catalog id=12>]

The catalogs returned were all the new subcatalogs created.

We can describe our catalog and see how the layout has changed:

In [68]:
merged_collection.describe()

* <Collection id=visual-cogs>
    * <Catalog id=2019>
        * <Catalog id=1>
          * <Item id=20190109_034415_103d>
          * <Item id=20190109_034414_103d>
          * <Item id=20190111_034459_0f3f>
          * <Item id=20190109_034416_103d>
          * <Item id=20190111_033800_0f46>
          * <Item id=20190111_034457_0f3f>
          * <Item id=20190111_033759_0f46>
          * <Item id=20190111_034458_0f3f>
          * <Item id=20190119_034511_1035>
        * <Catalog id=3>
          * <Item id=20190322_034910_0f12>
        * <Catalog id=2>
          * <Item id=20190212_033542_1054>
    * <Catalog id=2018>
        * <Catalog id=10>
          * <Item id=20181025_034420_0f4e>
        * <Catalog id=11>
          * <Item id=20181125_034649_0f28>
        * <Catalog id=9>
          * <Item id=20180924_034401_0f3f>
        * <Catalog id=12>
          * <Item id=20181227_034225_0f2b>


We can now save our catalog to a new location and move all the assets into our new catalog.

In [70]:
merged_collection = merged_collection.normalize_hrefs(os.path.join(STAC_DIR, 'final'))

In [71]:
merged_collection.normalize_hrefs(os.path.join(STAC_DIR, 'final'))
merged_collection.save()

Now that our catalog is in its final place, we'll move all the assets to be alongside their items.

In [73]:
final_catalog = stactools.core.move_all_assets(merged_collection, ignore_conflicts=True)

We can make all the asset HREFs relative to have a more readable STAC JSON.

In [75]:
final_catalog.make_all_asset_hrefs_relative()

In [76]:
final_catalog.save()

Our final STAC is ready to be used and browsed by `stac browse`